# Lexical framing: code to construct month-by-month tables of desired data from the decahose, stored as Parquet
Lexical framing project with Hang

In [1]:

import datetime
import boto3
import pyathena

In [4]:
# Assumes a file ~/.aws/credentials exists

session = boto3.Session(profile_name='lsm',)

In [10]:
frozen_creds = session.get_credentials().get_frozen_credentials()
conn = pyathena.connect(aws_access_key_id=frozen_creds.access_key,
                        aws_secret_access_key=frozen_creds.secret_key,
                        schema_name='lexical_framing',
                        s3_staging_dir='s3://lsm-data/lexical-framing/athena_cache/',
                        region_name='us-east-1')

In [11]:
# date_starts = ['2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01',
#                '2021-01-01','2021-02-01','2021-03-01','2021-04-01','2021-05-01','2021-06-01','2021-07-01']

# date_starts = ['2021-07-01','2021-08-01','2021-09-01','2021-10-01','2021-11-01','2021-12-01',
#                '2022-01-01','2022-02-01','2022-03-01','2022-04-01','2022-05-01','2022-06-01',
#                '2022-07-01','2022-08-01','2022-09-01','2022-10-01','2022-11-01','2022-12-01']

date_starts = ['2022-12-01','2023-01-01']


#date_starts = ['2020-07-01','2020-07-03']


In [12]:
cursor = conn.cursor()

for start_day_str,end_day_str in zip(date_starts[:-1],date_starts[1:]):
    print(f"Processing date range: [{start_day_str},{end_day_str}). Running query at time: {datetime.datetime.now()}")
    
    query1 = f"""
CREATE TABLE IF NOT EXISTS lexical_framing.us_deca_{start_day_str.replace('-','')}
WITH (
    format='PARQUET',
    write_compression = 'SNAPPY',
    external_location='s3://lsm-data/lexical-framing/us_deca/us_deca_{start_day_str.replace('-','')}/',
    partitioned_by = ARRAY['day']) 
AS
SELECT
  "split_part"("json_extract_scalar"(line, '$.id'), ':', 3) id
, "json_extract_scalar"(line, '$.postedTime') postedTime
, "json_extract_scalar"(line, '$.verb') verb

, "split_part"("json_extract_scalar"(line, '$.actor.id'), ':', 3) actor_id
, "json_extract_scalar"(line, '$.actor.preferredUsername') actor_preferredUsername
, "json_extract_scalar"(line, '$.actor.displayName') actor_displayName
, "json_extract_scalar"(line, '$.actor.summary') actor_summary
, "json_extract_scalar"(line, '$.actor.image') actor_image
, CAST("json_extract_scalar"(line, '$.actor.friendsCount') AS INTEGER) actor_friendsCount
, CAST("json_extract_scalar"(line, '$.actor.followersCount') AS INTEGER) actor_followersCount
, CAST("json_extract_scalar"(line, '$.actor.statusesCount') AS INTEGER) actor_statusesCount
, CAST("json_extract_scalar"(line, '$.actor.favoritesCount') AS INTEGER) actor_favoritesCount
, "json_extract_scalar"(line, '$.actor.location.displayName') actor_location_displayName

, "json_extract_scalar"(line, '$.twitter_lang') twitter_lang
, "json_extract_scalar"(line, '$.body') body
, "json_extract_scalar"(line, '$.long_object.body') longobj_body
, "json_extract_scalar"(line, '$.object.long_object.body') object_longobj_body
, day
FROM
  decahose.decahose_line
WHERE (
    (line <> '') AND 
    ("json_extract_scalar"(line, '$.objectType') = 'activity') AND
    ("json_extract_scalar"(line, '$.verb') = 'post') AND
    ("json_extract_scalar"(line, '$.twitter_lang') = 'en') AND
    (("json_extract_scalar"(line, '$.actor.location.displayName') <> '') OR 
        ("json_extract_scalar"(line, '$.actor.location.displayName') IS NOT NULL)) AND
    (day >= DATE '{start_day_str}' AND day < DATE '{end_day_str}')
    )
    """
#    print(query1)
    cursor.execute(query1)

Processing date range: [2022-12-01,2023-01-01). Running query at time: 2023-01-29 21:41:29.010188


In [13]:
cursor.close()

In [14]:
conn.close()